# 1. Getting Started
## An introduction to ice flow modeling with the Parallel Ice Sheet Model (PISM)

*Author: Andy Aschwanden*

In this tutorial, we will perform a few progonostic simulations of the Greenland Ice Sheet with climate forcing provided by the [Ice Sheet Model Intercomparison for CMIP6 (ISMIP6)](https://climate-cryosphere.org/about-ismip6/]).

In [ ]:
from functools import partial

from dask.diagnostics import ProgressBar
from pathlib import Path
import matplotlib.pylab as plt
from matplotlib import colors
import numpy as np
import xarray as xr
import cf_xarray.units  # otherwise we get a Parser error
import pint_xarray
import xskillscore as xs

from pism_tutorials.download import download_files
from pism_tutorials.plotting import register_colormaps
from pism_tutorials.processing import preprocess_nc
from pism_tutorials.utils import merge_dicts, dict2str, sort_dict_by_key

register_colormaps()
xr.set_options(keep_attrs=True)


In [ ]:
# The name of the PISM Cloud S3 bucket
bucket_name = "pism-cloud-data"

In [ ]:
tutorial_files = {"initial_state": {"path": "initial_states", "files": ["g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc"]}, 
                  "basal_heatflux": {"path": "bheatflux", "files": ["geothermal_heat_flow_map_10km.nc"]},
                  "climate": {"path": "ismip6", "files": ["MARv3.9_MIROC5-rcp26_climate_1960-2100_v1.nc", "MARv3.9_MIROC5-rcp85_climate_1960-2100_v1.nc", "MARv3.9_ERAI_climate_1978-2018_MEAN.nc"]},
                  "ocean": {"path": "ismip6", "files": ["MAR3.9_MIROC-ESM-CHEM_rcp26_ocean_1960-2100_v4.nc", "MAR3.9_MIROC-ESM-CHEM_rcp85_ocean_1960-2100_v4.nc"]},
                  "observations": {"path": "observations", "files": ["GRE_G1200_0000.nc"]},
                  "grid": {"path": "grids", "files": ["pism-bedmachine.nc"]}}

to_download = []
for forcing_type, forcing in tutorial_files.items():
    for f in forcing["files"]:
        d = Path(forcing["path"])
        d.mkdir(parents=True, exist_ok=True)
        p = d / Path(f)
        to_download.append(str(p))


In [ ]:
# Download files if not exist.
# Set "overwrite=True" to overwrite existing files.
download_files(bucket_name, to_download, overwrite=False)

In [ ]:
init_file = Path(tutorial_files["initial_state"]["path"]) / Path(tutorial_files["initial_state"]["files"][0])
init_ds = xr.open_dataset(init_file, decode_timedelta=True)

climate_files = Path(tutorial_files["climate"]["path"]).glob("*rcp*_climate_*.nc")
climate_ds = xr.open_mfdataset(climate_files,                       
                               preprocess=partial(preprocess_nc, regexp="_(.+?)_", dim="gcm"),
                               chunks="auto",
                               parallel=True,             
                               decode_cf=True,
                               decode_timedelta=True,
                               combine="nested",
                               concat_dim="gcm").sortby("gcm").squeeze()

climate_ds["climatic_mass_balance_anomaly"] = climate_ds["climatic_mass_balance_anomaly"].pint.quantify().pint.to("kg m-2 yr-1")

bhf_file = Path(tutorial_files["basal_heatflux"]["path"]) / Path(tutorial_files["basal_heatflux"]["files"][0])
bhf_ds = xr.open_dataset(bhf_file, decode_timedelta=True)

## Initial state

Generating initial states that are compatible with observations and our understanding of ice flow is still an active area of research. For the purposes of this tutorial we use an initial state adapted from {cite:t}`Aschwanden2022`.


In summary, the inital state was obtained by combining an energy field from simulation spanning a glacial cycle, present day ice sheet geometry, and basal fields calibrated using present day surface speeds.

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12, 4))
init_ds["usurf"].plot(ax=axs[0], cmap="bath_topo", vmin=-500, vmax=2000, extend="both")
init_ds["topg"].plot(ax=axs[1], cmap="bath_topo", vmin=-500, vmax=2000, extend="both")
init_ds["thk"].where(init_ds["thk"] > 0).plot(ax=axs[2], cmap="managua", vmax=3000, extend="max")
axs[0].set_title("ice surface elevation")
axs[1].set_title("sublacial topography")
axs[2].set_title("ice thickness")
for ax in axs:
    ax.set_axis_off()

Open surface speeds from the `ITS_LIVE` Greenland Mosaic and compare to simulated surface speeds.

In [ ]:
itslive_file = Path(tutorial_files["observations"]["path"]) / Path(tutorial_files["observations"]["files"][0])

itslive_ds = xr.open_dataset(itslive_file,
                             decode_timedelta=True,

)

speed_sim = init_ds["velsurf_mag"].squeeze()
speed_obs = itslive_ds["v"].squeeze()

fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12, 4))
speed_sim.plot(ax=axs[0], cmap="speed_colorblind", vmin=10, vmax=1500, extend="both")
speed_obs.plot(ax=axs[1], cmap="speed_colorblind", vmin=10, vmax=1500, extend="both")
(speed_sim-speed_obs).plot(ax=axs[2], cmap="RdBu_r", vmin=-250, vmax=250, extend="both")
axs[0].set_title("Simulated")
axs[1].set_title("Observed")
axs[2].set_title("Simulated - Observed")
for ax in axs:
    ax.set_axis_off()


mae = xs.mae(speed_sim, speed_obs, skipna=True)

print("Simulated - Observed:")
print("*" * 80)
print("")
print(f"Mean Absolute Error: {mae:.1f} m/yr")

## Climate forcing

Let's familiarize ourselves with the climate forcing

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 3))
climate_ds.sel({"time": slice("2015", None)})["ice_surface_temp_anomaly"].mean(dim=["x", "y"]).plot(ax=ax, hue="gcm")

In [ ]:
anomaly_2090s = climate_ds.sel({"time": slice("2090", "2099")}).mean(dim="time")

fig = anomaly_2090s["climatic_mass_balance_anomaly"].plot(col="gcm", vmin=-2500, vmax=2500, cmap="RdBu", extend="both", figsize=(8, 4)).fig
for ax in fig.axes[:2]:
    ax.set_axis_off()
fig.suptitle("Climatic Mass Balance Anomaly Mean 2090-2099")
fig.subplots_adjust(top=0.85, right=0.8)
fig = anomaly_2090s["ice_surface_temp_anomaly"].plot(col="gcm", cmap="Reds", vmin=0, vmax=20, extend="max", figsize=(8, 4)).fig
for ax in fig.axes[:2]:
    ax.set_axis_off()
fig.suptitle("Ice Surface Temperature Anomaly Mean 2090-2099")
fig.subplots_adjust(top=0.85, right=0.8)

## Setting up the simulation

At present, not all PISM functions are exposed via its python-API. We will thus use python dictionaries to set parameters, define the grid, choose the climate forcing, and so on. We will then translate the dictionaries into a `str` that we can run at the command line.

In [ ]:
resolution = "4500m"
output_dir = Path("test_run")
output_dir.mkdir(parents=True, exist_ok=True)

extra_file = ""
start = "2015-01-01"
end = "2099-12-31"

input_params = {
    "bootstrap": "",
    "i": "initial_states/g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc",
    "input.regrid.file": "initial_states/g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc",
    "input.regrid.vars": "litho_temp,enthalpy,age,tillwat,bmelt,ice_area_specific_volume,thk"
}

grid_params = {
    "grid.dx": resolution,
    "grid.dy": resolution,
    "grid.Mz": 101,
    "grid.Lz": 4000,
    "grid.Mbz": 11,
    "grid.Lbz": 1000,
    "input.regrid.vars": "litho_temp,enthalpy,age,tillwat,bmelt,ice_area_specific_volume,thk"
}

time_params = {
    "time.start": start,
    "time.end": end,
    "time.calendar": "standard",
    "time_stepping.skip.enabled": "",
    "time_stepping.skip.max": 100
}

climate_params = {
    "surface.models": "ismip6",
    "surface.ismip6.reference_file": "ismip6/MARv3.9_ERAI_climate_1978-2018_MEAN.nc",
    "surface.ismip6.file": ""
}

stress_balance = {
    "stress.balance": "ssa+sia",
    "stress_balance.calving_front_stress_bc": "",  
    "stress_balance.ice_free_thickness_standard": 5,
    "stress_balance.sia.bed_smoother.range": resolution[:-1], 
    "stress_balance.sia.enhancement_factor": 2.608046,
    "stress_balance.sia.flow_law": "gpbld",
    "stress_balance.ssa.Glen_exponent": 3.309718,
    "stress_balance.ssa.enhancement_factor": 1.0,
    "basal_resistance.pseudo_plastic.enabled": "",
    "basal_resistance.pseudo_plastic.q": 0.7508221,
    "basal_yield_stress.mohr_coulomb.till_effective_fraction_overburden": 0.01845403,
    "basal_yield_stress.mohr_coulomb.topg_to_phi.enabled": "",
    "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_max": 42.79528,
    "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_min": 7.193718, 
    "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_max": 243.8239, 
    "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_min": -369.6359, 
}

output_params = {
    "output.extra.file": extra_file,
    "output.extra.times": "yearly",
    "output.extra.vars": "velsurf_mag,usurf,thk,climatic_mass_balance,ice_surface_temp,mask,mass_fluxes,ice_mass_transport_across_grounding_line,ice_mass",
    "output.size": "none"  # do not write a state file
}

run_dict = merge_dicts(input_params, grid_params, time_params, stress_balance,climate_params, output_params)

## Running PISM

We perform two simulations, one using climate anomalies based on RCP 2.6 and other based on RCP 8.5.

In [ ]:
climate_files = ["ismip6/MARv3.9_MIROC5-rcp26_climate_1960-2100_v1.nc", "ismip6/MARv3.9_MIROC5-rcp85_climate_1960-2100_v1.nc"]

n = 8
for rcp, climate_file in zip([26, 85], climate_files):
    extra_file = Path(output_dir) / f"spatial_g{resolution}_rcp_{rcp}_{start}_{end}.nc"
    run_dict.update({"surface.ismip6.file": climate_file, "output.extra.file": extra_file})
    run_str = dict2str(sort_dict_by_key(run_dict))
    cmd = f"mpirun -np {n} pism " + run_str
    ! $cmd

## Results

We collect the two files containing the spatial variables and open the files with `xarray`.

In [ ]:
spatial_files = Path(output_dir).glob(f"spatial_g{resolution}_rcp_*{start}_{end}.nc")
spatial_ds = xr.open_mfdataset(spatial_files,                       
                               preprocess=partial(preprocess_nc, regexp="rcp_(.+?)_", dim="rcp"),
                               chunks="auto",
                               parallel=True,             
                               decode_cf=True,
                               decode_timedelta=True,
                               combine="nested",
                               concat_dim="rcp").sortby("rcp").squeeze()


### Scalar mass change

Let's compare the total mass change from 2015-2100 for RCP 2.6 and RCP 8.5.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 3))
ice_mass = spatial_ds.sel({"time": slice("2015", None)})["ice_mass"].sum(dim=["x", "y"]).pint.quantify().pint.to("Gt")
ice_mass_scaled = ice_mass
ice_mass_scaled -= ice_mass.isel({"time": 0})
ice_mass_scaled.plot(ax=ax, hue="rcp")

Both simulations actually *gain* mass until 2050, after which the RCP 8.5-based simulations starts to lose mass yet ending up with more mass in 2100 than 2015. This is clearly wrong.

So what are the reasons?

1. The grid resolution is too low: We are not *resolving* outlet glaciers, resulting in the simulation under-estimating `grounding line flux`.
2. The *initial state* may have inconsistencies that lead to unphysical transients.

We can address *grid resolution* by using HPC/HTC.

## References
```{bibliography} references.bib
```